# テナースプレッド分析

LME銅のテナースプレッド（限月間価格差）分析を行います。これは本プロジェクトの中核となる分析です。

## 分析内容
- Cash/3M スプレッド分析
- 連続限月間スプレッド分析
- コンタンゴ・バックワーデーション判定
- スプレッド変動要因分析（在庫、市場指標との関係）
- 季節性分析

In [ ]:
# 必要なライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# 統計・機械学習ライブラリ
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# 日本語フォント設定
plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# 自作モジュールのインポート
import sys
sys.path.append('../config')
from data_utils import DataFetcher
from database_config import VISUALIZATION_CONFIG

# DataFetcherの初期化
fetcher = DataFetcher()

print("📐 テナースプレッド分析ノートブック")
print("=" * 50)

## 1. データ取得・前処理

In [ ]:
# テナースプレッド分析用データの取得
print("📥 テナーデータ取得中...")

# 限月別価格データ
tenor_data = fetcher.get_tenor_spread_data(days=365)
print(f"✅ テナーデータ: {len(tenor_data)} レコード")

# 在庫データ（スプレッド要因分析用）
lme_inventory = fetcher.get_lme_inventory(days=365)
inventory_total = lme_inventory.groupby('ReportDate')['TotalStock'].sum().reset_index()
print(f"✅ 在庫データ: {len(inventory_total)} レコード")

# 市場指標データ（金利など）
market_indicators = fetcher.get_market_indicators(days=365, categories=['Interest Rate'])
print(f"✅ 市場指標データ: {len(market_indicators)} レコード")

print(f"\n📅 期間: {tenor_data['TradeDate'].min()} ～ {tenor_data['TradeDate'].max()}")
print(f"🏢 取引所: {tenor_data['ExchangeCode'].unique()}")
print(f"📈 限月タイプ: {tenor_data['TenorTypeName'].unique()}")

# データの概要表示
tenor_data.head()

In [ ]:
# LMEテナーデータの整理
lme_tenor = tenor_data[tenor_data['ExchangeCode'] == 'LME'].copy()

# ピボットテーブルの作成
lme_prices_pivot = lme_tenor.pivot_table(
    index='TradeDate',
    columns='TenorTypeName',
    values='LastPrice',
    aggfunc='first'
).fillna(method='ffill')

print(f"📊 LME限月別価格データ: {len(lme_prices_pivot)} 日間")
print(f"📈 利用可能な限月: {list(lme_prices_pivot.columns)}")

# 利用可能なカラムの確認
available_tenors = lme_prices_pivot.columns.tolist()
print(f"\n🔍 検出された限月タイプ: {available_tenors}")

lme_prices_pivot.describe()

## 2. スプレッド計算

In [ ]:
# スプレッドデータフレームの初期化
spreads = pd.DataFrame(index=lme_prices_pivot.index)

# 1. Cash/3M スプレッド（最重要）
if 'Cash' in available_tenors and '3M Futures' in available_tenors:
    spreads['Cash_3M'] = lme_prices_pivot['3M Futures'] - lme_prices_pivot['Cash']
    print("✅ Cash/3M スプレッド計算完了")

# 2. Generic 1st/2nd スプレッド
if 'Generic 1st Future' in available_tenors and 'Generic 2nd Future' in available_tenors:
    spreads['1st_2nd'] = lme_prices_pivot['Generic 2nd Future'] - lme_prices_pivot['Generic 1st Future']
    print("✅ 1st/2nd スプレッド計算完了")

# 3. Generic 2nd/3rd スプレッド
if 'Generic 2nd Future' in available_tenors and 'Generic 3rd Future' in available_tenors:
    spreads['2nd_3rd'] = lme_prices_pivot['Generic 3rd Future'] - lme_prices_pivot['Generic 2nd Future']
    print("✅ 2nd/3rd スプレッド計算完了")

# 4. Cash/1st Generic スプレッド
if 'Cash' in available_tenors and 'Generic 1st Future' in available_tenors:
    spreads['Cash_1st'] = lme_prices_pivot['Generic 1st Future'] - lme_prices_pivot['Cash']
    print("✅ Cash/1st スプレッド計算完了")

# 5. 長期スプレッド（1st/3rd）
if 'Generic 1st Future' in available_tenors and 'Generic 3rd Future' in available_tenors:
    spreads['1st_3rd'] = lme_prices_pivot['Generic 3rd Future'] - lme_prices_pivot['Generic 1st Future']
    print("✅ 1st/3rd スプレッド計算完了")

# スプレッドデータの確認
spreads = spreads.dropna()
print(f"\n📊 スプレッドデータ: {len(spreads)} 日間")
print(f"📈 計算されたスプレッド: {list(spreads.columns)}")

# スプレッド統計
print("\n📋 スプレッド統計 (USD/MT):")
spreads.describe()

## 3. Cash/3M スプレッド分析（重要）

In [ ]:
# Cash/3M スプレッドの詳細分析
if 'Cash_3M' in spreads.columns:
    cash_3m = spreads['Cash_3M'].copy()
    
    # 基本統計
    print(f"📊 Cash/3M スプレッド分析")
    print(f"平均: {cash_3m.mean():.2f} USD/MT")
    print(f"標準偏差: {cash_3m.std():.2f} USD/MT")
    print(f"最小値: {cash_3m.min():.2f} USD/MT")
    print(f"最大値: {cash_3m.max():.2f} USD/MT")
    
    # コンタンゴ・バックワーデーション判定
    contango_days = (cash_3m > 0).sum()
    backwardation_days = (cash_3m < 0).sum()
    total_days = len(cash_3m)
    
    print(f"\n🔄 市場構造分析:")
    print(f"コンタンゴ（正スプレッド）: {contango_days} 日 ({contango_days/total_days*100:.1f}%)")
    print(f"バックワーデーション（負スプレッド）: {backwardation_days} 日 ({backwardation_days/total_days*100:.1f}%)")
    
    # 最新状況
    latest_spread = cash_3m.iloc[-1]
    market_structure = "コンタンゴ" if latest_spread > 0 else "バックワーデーション"
    print(f"\n📅 最新状況: {latest_spread:.2f} USD/MT ({market_structure})")
    
    # 30日移動平均
    cash_3m_ma30 = cash_3m.rolling(window=30).mean()
    cash_3m_std30 = cash_3m.rolling(window=30).std()
    
    # Cash/3M スプレッド可視化
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Cash/3M スプレッド推移', 'スプレッド分布', '移動統計', 'Z-Score分析'),
        vertical_spacing=0.1
    )
    
    # 1. スプレッド推移
    fig.add_trace(
        go.Scatter(
            x=cash_3m.index,
            y=cash_3m.values,
            mode='lines',
            name='Cash/3M スプレッド',
            line=dict(color='blue', width=1),
            hovertemplate='日付: %{x}<br>スプレッド: %{y:.2f} USD/MT<extra></extra>'
        ),
        row=1, col=1
    )
    
    # 30日移動平均
    fig.add_trace(
        go.Scatter(
            x=cash_3m_ma30.index,
            y=cash_3m_ma30.values,
            mode='lines',
            name='30日移動平均',
            line=dict(color='red', width=2),
            hovertemplate='日付: %{x}<br>MA30: %{y:.2f} USD/MT<extra></extra>'
        ),
        row=1, col=1
    )
    
    # ゼロライン
    fig.add_hline(y=0, line_dash="dash", line_color="black", opacity=0.5, row=1, col=1)
    
    # 2. 分布
    fig.add_trace(
        go.Histogram(
            x=cash_3m.values,
            nbinsx=50,
            name='スプレッド分布',
            marker_color='lightblue',
            opacity=0.7
        ),
        row=1, col=2
    )
    
    # 3. 移動標準偏差
    fig.add_trace(
        go.Scatter(
            x=cash_3m_std30.index,
            y=cash_3m_std30.values,
            mode='lines',
            name='30日移動標準偏差',
            line=dict(color='orange', width=2)
        ),
        row=2, col=1
    )
    
    # 4. Z-Score（標準化スプレッド）
    z_score = (cash_3m - cash_3m_ma30) / cash_3m_std30
    fig.add_trace(
        go.Scatter(
            x=z_score.index,
            y=z_score.values,
            mode='lines',
            name='Z-Score',
            line=dict(color='green', width=1)
        ),
        row=2, col=2
    )
    
    # ±2σライン
    fig.add_hline(y=2, line_dash="dash", line_color="red", opacity=0.5, row=2, col=2)
    fig.add_hline(y=-2, line_dash="dash", line_color="red", opacity=0.5, row=2, col=2)
    fig.add_hline(y=0, line_dash="dash", line_color="black", opacity=0.5, row=2, col=2)
    
    # レイアウト設定
    fig.update_layout(
        height=800,
        title_text="🔸 Cash/3M スプレッド詳細分析",
        title_x=0.5
    )
    
    fig.update_xaxes(title_text="日付", row=1, col=1)
    fig.update_xaxes(title_text="スプレッド (USD/MT)", row=1, col=2)
    fig.update_xaxes(title_text="日付", row=2, col=1)
    fig.update_xaxes(title_text="日付", row=2, col=2)
    
    fig.update_yaxes(title_text="スプレッド (USD/MT)", row=1, col=1)
    fig.update_yaxes(title_text="頻度", row=1, col=2)
    fig.update_yaxes(title_text="標準偏差 (USD/MT)", row=2, col=1)
    fig.update_yaxes(title_text="Z-Score", row=2, col=2)
    
    fig.show()
else:
    print("⚠️ Cash/3M スプレッドデータが利用できません")

## 4. 複数スプレッドの比較分析

In [ ]:
# 全スプレッドの推移比較
if len(spreads.columns) > 1:
    # 正規化スプレッド（Z-Score）
    normalized_spreads = spreads.apply(lambda x: (x - x.mean()) / x.std())
    
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(14, 12))
    
    # 1. 生スプレッド
    for col in spreads.columns:
        ax1.plot(spreads.index, spreads[col], label=col, linewidth=2)
    
    ax1.set_title('🔸 全スプレッド推移（生データ）', fontsize=16, fontweight='bold')
    ax1.set_ylabel('スプレッド (USD/MT)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    
    # 2. 正規化スプレッド
    for col in normalized_spreads.columns:
        ax2.plot(normalized_spreads.index, normalized_spreads[col], label=col, linewidth=2)
    
    ax2.set_title('🔸 全スプレッド推移（正規化）', fontsize=16, fontweight='bold')
    ax2.set_ylabel('正規化スプレッド (Z-Score)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    
    # 3. スプレッド相関ヒートマップ
    spread_corr = spreads.corr()
    im = ax3.imshow(spread_corr.values, cmap='coolwarm', aspect='auto', vmin=-1, vmax=1)
    ax3.set_xticks(range(len(spread_corr.columns)))
    ax3.set_yticks(range(len(spread_corr.columns)))
    ax3.set_xticklabels(spread_corr.columns, rotation=45)
    ax3.set_yticklabels(spread_corr.columns)
    ax3.set_title('🔸 スプレッド相関マトリックス', fontsize=16, fontweight='bold')
    
    # 相関値をテキストで表示
    for i in range(len(spread_corr.columns)):
        for j in range(len(spread_corr.columns)):
            text = ax3.text(j, i, f'{spread_corr.iloc[i, j]:.2f}',
                           ha="center", va="center", color="black", fontweight='bold')
    
    plt.colorbar(im, ax=ax3, label='相関係数')
    plt.tight_layout()
    plt.show()
    
    # 相関分析結果
    print("🔗 スプレッド間相関分析:")
    print(spread_corr)
else:
    print("⚠️ 複数のスプレッドデータが利用できません")

## 5. スプレッド変動要因分析

In [ ]:
# スプレッドと在庫の関係分析
if 'Cash_3M' in spreads.columns and len(inventory_total) > 0:
    # データ統合
    spread_factors = pd.DataFrame()
    spread_factors['Date'] = spreads.index
    spread_factors['Cash_3M_Spread'] = spreads['Cash_3M'].values
    
    # 在庫データとマージ
    inventory_total['ReportDate'] = pd.to_datetime(inventory_total['ReportDate'])
    spread_factors = spread_factors.set_index('Date')
    inventory_series = inventory_total.set_index('ReportDate')['TotalStock']
    
    spread_factors = spread_factors.join(inventory_series.rename('Inventory'), how='inner')
    spread_factors = spread_factors.dropna()
    
    if len(spread_factors) > 30:  # 十分なデータがある場合
        # 在庫・スプレッド相関
        inventory_spread_corr = spread_factors['Cash_3M_Spread'].corr(spread_factors['Inventory'])
        print(f"📊 在庫・Cash/3Mスプレッド相関: {inventory_spread_corr:.3f}")
        
        # 在庫レベル別スプレッド分析
        # 在庫を四分位で分類
        spread_factors['Inventory_Quartile'] = pd.qcut(spread_factors['Inventory'], 
                                                      q=4, labels=['Q1(低)', 'Q2(中低)', 'Q3(中高)', 'Q4(高)'])
        
        quartile_stats = spread_factors.groupby('Inventory_Quartile')['Cash_3M_Spread'].agg(['mean', 'std', 'count'])
        print("\n📈 在庫レベル別スプレッド統計:")
        print(quartile_stats)
        
        # 可視化
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('在庫 vs スプレッド時系列', '在庫 vs スプレッド散布図', '在庫レベル別スプレッド分布', '在庫・スプレッド移動相関'),
            specs=[[{"secondary_y": True}, {"secondary_y": False}],
                   [{"secondary_y": False}, {"secondary_y": False}]],
            vertical_spacing=0.1
        )
        
        # 1. 時系列（二軸）
        fig.add_trace(
            go.Scatter(
                x=spread_factors.index,
                y=spread_factors['Cash_3M_Spread'],
                mode='lines',
                name='Cash/3M スプレッド',
                line=dict(color='blue', width=2),
                yaxis='y'
            ),
            row=1, col=1
        )
        
        fig.add_trace(
            go.Scatter(
                x=spread_factors.index,
                y=spread_factors['Inventory'],
                mode='lines',
                name='LME在庫',
                line=dict(color='red', width=2),
                yaxis='y2'
            ),
            row=1, col=1
        )
        
        # 2. 散布図
        fig.add_trace(
            go.Scatter(
                x=spread_factors['Inventory'],
                y=spread_factors['Cash_3M_Spread'],
                mode='markers',
                name='在庫 vs スプレッド',
                marker=dict(color='green', size=5, opacity=0.6),
                hovertemplate='在庫: %{x:,.0f} トン<br>' +
                             'スプレッド: %{y:.2f} USD/MT<extra></extra>'
            ),
            row=1, col=2
        )
        
        # 回帰線
        z = np.polyfit(spread_factors['Inventory'], spread_factors['Cash_3M_Spread'], 1)
        p = np.poly1d(z)
        x_reg = np.linspace(spread_factors['Inventory'].min(), spread_factors['Inventory'].max(), 100)
        
        fig.add_trace(
            go.Scatter(
                x=x_reg,
                y=p(x_reg),
                mode='lines',
                name=f'回帰線 (r={inventory_spread_corr:.3f})',
                line=dict(color='red', dash='dash')
            ),
            row=1, col=2
        )
        
        # 3. ボックスプロット（在庫レベル別）
        for i, quartile in enumerate(['Q1(低)', 'Q2(中低)', 'Q3(中高)', 'Q4(高)']):
            quartile_data = spread_factors[spread_factors['Inventory_Quartile'] == quartile]['Cash_3M_Spread']
            fig.add_trace(
                go.Box(
                    y=quartile_data,
                    name=quartile,
                    boxpoints='outliers'
                ),
                row=2, col=1
            )
        
        # 4. 移動相関
        rolling_corr = spread_factors['Cash_3M_Spread'].rolling(window=30).corr(spread_factors['Inventory'])
        fig.add_trace(
            go.Scatter(
                x=rolling_corr.index,
                y=rolling_corr.values,
                mode='lines',
                name='30日移動相関',
                line=dict(color='purple', width=2)
            ),
            row=2, col=2
        )
        
        fig.add_hline(y=0, line_dash="dash", line_color="black", opacity=0.5, row=2, col=2)
        
        # レイアウト設定
        fig.update_layout(
            height=800,
            title_text="🔸 スプレッド変動要因分析（在庫との関係）",
            title_x=0.5
        )
        
        # 軸設定
        fig.update_xaxes(title_text="日付", row=1, col=1)
        fig.update_xaxes(title_text="在庫量 (トン)", row=1, col=2)
        fig.update_xaxes(title_text="在庫レベル", row=2, col=1)
        fig.update_xaxes(title_text="日付", row=2, col=2)
        
        fig.update_yaxes(title_text="スプレッド (USD/MT)", secondary_y=False, row=1, col=1)
        fig.update_yaxes(title_text="在庫量 (トン)", secondary_y=True, row=1, col=1)
        fig.update_yaxes(title_text="スプレッド (USD/MT)", row=1, col=2)
        fig.update_yaxes(title_text="スプレッド (USD/MT)", row=2, col=1)
        fig.update_yaxes(title_text="相関係数", row=2, col=2)
        
        fig.show()
    else:
        print("⚠️ 在庫・スプレッド分析に十分なデータがありません")
else:
    print("⚠️ スプレッド・在庫データが利用できません")

## 6. 季節性・周期性分析

In [ ]:
# Cash/3M スプレッドの季節性分析
if 'Cash_3M' in spreads.columns:
    seasonal_data = spreads['Cash_3M'].copy()
    seasonal_df = pd.DataFrame({
        'Spread': seasonal_data,
        'Month': seasonal_data.index.month,
        'Quarter': seasonal_data.index.quarter,
        'DayOfWeek': seasonal_data.index.dayofweek,
        'Year': seasonal_data.index.year
    })
    
    # 月別統計
    monthly_stats = seasonal_df.groupby('Month')['Spread'].agg(['mean', 'std', 'count'])
    monthly_stats.index = ['1月', '2月', '3月', '4月', '5月', '6月', 
                          '7月', '8月', '9月', '10月', '11月', '12月']
    
    # 四半期別統計
    quarterly_stats = seasonal_df.groupby('Quarter')['Spread'].agg(['mean', 'std', 'count'])
    quarterly_stats.index = ['Q1', 'Q2', 'Q3', 'Q4']
    
    # 曜日別統計
    weekday_stats = seasonal_df.groupby('DayOfWeek')['Spread'].agg(['mean', 'std', 'count'])
    weekday_stats.index = ['月', '火', '水', '木', '金', '土', '日']
    
    print("📅 季節性分析結果:")
    print("\n🗓️ 月別スプレッド統計:")
    print(monthly_stats)
    print("\n📊 四半期別スプレッド統計:")
    print(quarterly_stats)
    print("\n📆 曜日別スプレッド統計:")
    print(weekday_stats)
    
    # 季節性可視化
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    # 1. 月別ボックスプロット
    seasonal_df.boxplot(column='Spread', by='Month', ax=axes[0,0])
    axes[0,0].set_title('🔸 月別スプレッド分布')
    axes[0,0].set_xlabel('月')
    axes[0,0].set_ylabel('スプレッド (USD/MT)')
    axes[0,0].grid(True, alpha=0.3)
    
    # 2. 四半期別バープロット
    quarterly_stats['mean'].plot(kind='bar', ax=axes[0,1], color='skyblue', alpha=0.7)
    axes[0,1].set_title('🔸 四半期別平均スプレッド')
    axes[0,1].set_xlabel('四半期')
    axes[0,1].set_ylabel('平均スプレッド (USD/MT)')
    axes[0,1].tick_params(axis='x', rotation=0)
    axes[0,1].grid(True, alpha=0.3)
    
    # 3. 曜日別バープロット
    weekday_stats['mean'].plot(kind='bar', ax=axes[1,0], color='lightgreen', alpha=0.7)
    axes[1,0].set_title('🔸 曜日別平均スプレッド')
    axes[1,0].set_xlabel('曜日')
    axes[1,0].set_ylabel('平均スプレッド (USD/MT)')
    axes[1,0].tick_params(axis='x', rotation=0)
    axes[1,0].grid(True, alpha=0.3)
    
    # 4. 年次推移
    yearly_stats = seasonal_df.groupby('Year')['Spread'].mean()
    yearly_stats.plot(kind='line', ax=axes[1,1], marker='o', linewidth=2, markersize=6)
    axes[1,1].set_title('🔸 年次平均スプレッド推移')
    axes[1,1].set_xlabel('年')
    axes[1,1].set_ylabel('平均スプレッド (USD/MT)')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 統計的検定（月別の違いの有意性）
    from scipy.stats import f_oneway
    
    monthly_groups = [seasonal_df[seasonal_df['Month'] == month]['Spread'].values 
                     for month in range(1, 13)]
    monthly_groups = [group for group in monthly_groups if len(group) > 0]  # 空のグループを除外
    
    if len(monthly_groups) >= 2:
        f_stat, p_value = f_oneway(*monthly_groups)
        print(f"\n🧪 月別スプレッドの分散分析:")
        print(f"F統計量: {f_stat:.3f}")
        print(f"p値: {p_value:.6f}")
        print(f"有意差: {'あり' if p_value < 0.05 else 'なし'} (α=0.05)")
else:
    print("⚠️ 季節性分析用データが利用できません")

## 7. スプレッド予測モデル（簡易版）

In [ ]:
# 簡易スプレッド予測モデル
if 'Cash_3M' in spreads.columns and len(spread_factors) > 50:
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import mean_squared_error, mean_absolute_error
    
    # 特徴量エンジニアリング
    model_data = spread_factors.copy()
    
    # ラグ特徴量
    for lag in [1, 2, 3, 5, 10]:
        model_data[f'Spread_lag_{lag}'] = model_data['Cash_3M_Spread'].shift(lag)
        model_data[f'Inventory_lag_{lag}'] = model_data['Inventory'].shift(lag)
    
    # 移動平均特徴量
    for window in [5, 10, 20]:
        model_data[f'Spread_ma_{window}'] = model_data['Cash_3M_Spread'].rolling(window=window).mean()
        model_data[f'Inventory_ma_{window}'] = model_data['Inventory'].rolling(window=window).mean()
    
    # 時系列特徴量
    model_data['Month'] = model_data.index.month
    model_data['Quarter'] = model_data.index.quarter
    model_data['DayOfWeek'] = model_data.index.dayofweek
    
    # 欠損値除去
    model_data = model_data.dropna()
    
    if len(model_data) > 30:
        # 特徴量とターゲットの分離
        feature_cols = [col for col in model_data.columns if col != 'Cash_3M_Spread']
        X = model_data[feature_cols]
        y = model_data['Cash_3M_Spread']
        
        # 訓練・テストデータ分割（時系列を考慮）
        split_point = int(len(model_data) * 0.8)
        X_train, X_test = X.iloc[:split_point], X.iloc[split_point:]
        y_train, y_test = y.iloc[:split_point], y.iloc[split_point:]
        
        # モデル訓練
        rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
        rf_model.fit(X_train, y_train)
        
        # 予測
        y_pred = rf_model.predict(X_test)
        
        # 評価指標
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        print("🤖 スプレッド予測モデル評価:")
        print(f"平均二乗誤差 (MSE): {mse:.3f}")
        print(f"平均絶対誤差 (MAE): {mae:.3f}")
        print(f"決定係数 (R²): {r2:.3f}")
        
        # 特徴量重要度
        feature_importance = pd.DataFrame({
            'feature': X.columns,
            'importance': rf_model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        print("\n📊 特徴量重要度（上位10位）:")
        print(feature_importance.head(10))
        
        # 予測結果可視化
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))
        
        # 予測 vs 実測
        test_dates = model_data.index[split_point:]
        ax1.plot(test_dates, y_test.values, label='実測値', linewidth=2, color='blue')
        ax1.plot(test_dates, y_pred, label='予測値', linewidth=2, color='red', alpha=0.7)
        ax1.set_title('🔸 スプレッド予測結果', fontsize=16, fontweight='bold')
        ax1.set_ylabel('Cash/3M スプレッド (USD/MT)')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # 特徴量重要度
        top_features = feature_importance.head(10)
        ax2.barh(range(len(top_features)), top_features['importance'].values, alpha=0.7)
        ax2.set_yticks(range(len(top_features)))
        ax2.set_yticklabels(top_features['feature'].values)
        ax2.set_title('🔸 特徴量重要度（上位10位）', fontsize=16, fontweight='bold')
        ax2.set_xlabel('重要度')
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    else:
        print("⚠️ 予測モデル構築に十分なデータがありません")
else:
    print("⚠️ 予測モデル用データが利用できません")

## 8. 分析サマリー

In [ ]:
# テナースプレッド分析結果のサマリー
print("📋 テナースプレッド分析サマリー")
print("=" * 50)

# 計算されたスプレッド
print(f"\n📊 分析対象スプレッド: {list(spreads.columns)}")
print(f"📅 分析期間: {spreads.index.min()} ～ {spreads.index.max()}")
print(f"📈 データ数: {len(spreads)} 日間")

# Cash/3M スプレッド詳細
if 'Cash_3M' in spreads.columns:
    cash_3m = spreads['Cash_3M']
    latest_spread = cash_3m.iloc[-1]
    market_structure = "コンタンゴ" if latest_spread > 0 else "バックワーデーション"
    
    print(f"\n💰 Cash/3M スプレッド分析:")
    print(f"  最新値: {latest_spread:.2f} USD/MT ({market_structure})")
    print(f"  平均値: {cash_3m.mean():.2f} USD/MT")
    print(f"  標準偏差: {cash_3m.std():.2f} USD/MT")
    print(f"  最小値: {cash_3m.min():.2f} USD/MT")
    print(f"  最大値: {cash_3m.max():.2f} USD/MT")
    
    # 市場構造分析
    contango_days = (cash_3m > 0).sum()
    backwardation_days = (cash_3m < 0).sum()
    total_days = len(cash_3m)
    
    print(f"\n🔄 市場構造分析:")
    print(f"  コンタンゴ: {contango_days} 日 ({contango_days/total_days*100:.1f}%)")
    print(f"  バックワーデーション: {backwardation_days} 日 ({backwardation_days/total_days*100:.1f}%)")

# スプレッド相関
if len(spreads.columns) > 1:
    print("\n🔗 スプレッド間相関:")
    spread_corr = spreads.corr()
    for i in range(len(spread_corr.columns)):
        for j in range(i+1, len(spread_corr.columns)):
            corr_val = spread_corr.iloc[i, j]
            print(f"  {spread_corr.columns[i]} vs {spread_corr.columns[j]}: {corr_val:.3f}")

# 在庫・スプレッド関係
if 'spread_factors' in locals() and len(spread_factors) > 0:
    print(f"\n📦 在庫・スプレッド関係:")
    print(f"  相関係数: {inventory_spread_corr:.3f}")
    
    # 最新30日の相関
    recent_corr = spread_factors.tail(30)['Cash_3M_Spread'].corr(spread_factors.tail(30)['Inventory'])
    print(f"  最新30日相関: {recent_corr:.3f}")

# 季節性分析結果
if 'monthly_stats' in locals():
    print("\n📅 季節性分析:")
    
    # 最も高い/低い月
    highest_month = monthly_stats['mean'].idxmax()
    lowest_month = monthly_stats['mean'].idxmin()
    
    print(f"  スプレッド最高月: {highest_month} ({monthly_stats.loc[highest_month, 'mean']:.2f} USD/MT)")
    print(f"  スプレッド最低月: {lowest_month} ({monthly_stats.loc[lowest_month, 'mean']:.2f} USD/MT)")
    
    if 'p_value' in locals():
        print(f"  月別有意差: {'あり' if p_value < 0.05 else 'なし'} (p={p_value:.4f})")

# 予測モデル結果
if 'r2' in locals():
    print("\n🤖 予測モデル性能:")
    print(f"  決定係数 (R²): {r2:.3f}")
    print(f"  平均絶対誤差: {mae:.3f} USD/MT")
    
    if 'feature_importance' in locals():
        top_feature = feature_importance.iloc[0]['feature']
        top_importance = feature_importance.iloc[0]['importance']
        print(f"  最重要特徴量: {top_feature} ({top_importance:.3f})")

# データ品質
print("\n📊 データ品質:")
print(f"  テナーデータレコード数: {len(tenor_data):,}")
print(f"  取引所数: {len(tenor_data['ExchangeCode'].unique())}")
print(f"  限月タイプ数: {len(tenor_data['TenorTypeName'].unique())}")
print(f"  欠損値: {spreads.isna().sum().sum()}")

# 重要な発見事項
print("\n🔍 重要な発見事項:")
if 'Cash_3M' in spreads.columns:
    recent_volatility = cash_3m.tail(30).std()
    historical_volatility = cash_3m.std()
    
    if recent_volatility > historical_volatility * 1.5:
        print("  ⚠️ 最近のスプレッドボラティリティが高い")
    elif recent_volatility < historical_volatility * 0.5:
        print("  📉 最近のスプレッドボラティリティが低い")
    
    if abs(latest_spread) > cash_3m.std() * 2:
        print(f"  🚨 現在のスプレッドが異常値レベル（±2σ超）")

print("\n✅ テナースプレッド分析完了")